This notebook generates the umap embeddings of the images in the dataset. The embeddings are saved in a parquet file.

In [1]:
import argparse
import pathlib

import numpy as np
import pandas as pd
import umap

In [ ]:
# set the arg parser
parser = argparse.ArgumentParser(description="UMAP on a matrix")

parser.add_argument("--data_mode", type=str, default="CP", help="data mode to use")

# get the args
args = parser.parse_args()

# set data mode to either "CP" or "scDINO" or "combined"
data_mode = args.data_mode

In [3]:
# set the paths to the data
CP_fs_sc_profiles_path = pathlib.Path(
    "../../data/CP_feature_select/profiles/features_selected_profile.parquet"
).resolve(strict=True)
combined_profiles_path = pathlib.Path(
    "../../data/CP_scDINO_features/combined_CP_scDINO_norm_fs.parquet"
).resolve(strict=True)
scDINO_fs_profiles_path = pathlib.Path(
    "../../data/scDINO/CLS_features_annotated_normalized_feature_selected.parquet"
).resolve(strict=True)

output_path = pathlib.Path(f"../../data/umap/").resolve()
output_path.mkdir(parents=True, exist_ok=True)

In [4]:
if data_mode == "CP":
    # read the data
    profiles_df = pd.read_parquet(CP_fs_sc_profiles_path)
elif data_mode == "combined":
    # read the data
    profiles_df = pd.read_parquet(combined_profiles_path)
elif data_mode == "scDINO":
    # read the data
    profiles_df = pd.read_parquet(scDINO_fs_profiles_path)
else:
    raise ValueError("data_mode must be either 'CP' or 'scDINO' or 'combined'")
print(profiles_df.shape)
# show all columns
pd.set_option("display.max_columns", None)
profiles_df.head()

(188065, 1546)


Metadata_image_path Metadata_Well  \
0  ../0.pre-process_images/data/processed_images/...          C-02   
1  ../0.pre-process_images/data/processed_images/...          C-02   
2  ../0.pre-process_images/data/processed_images/...          C-02   
3  ../0.pre-process_images/data/processed_images/...          C-02   
4  ../0.pre-process_images/data/processed_images/...          C-02   

  Metadata_FOV Metadata_Time Metadata_ImageNumber  \
0         0001          0001                    1   
1         0001          0001                    1   
2         0001          0001                    1   
3         0001          0001                    1   
4         0001          0001                    1   

  Metadata_Nuclei_Number_Object_Number Metadata_original_index  \
0                                  102                  111256   
1                                  106                  111257   
2                                  121                  111258   
3                                  127                  111259   
4                                   13                  111241   

  Metadata_compound  Metadata_dose Metadata_control  \
0     Staurosporine            0.0         negative   
1     Staurosporine            0.0         negative   
2     Staurosporine            0.0         negative   
3     Staurosporine            0.0         negative   
4     Staurosporine            0.0         negative   

   channel488-1_cls_feature_0  channel488-1_cls_feature_1  \
0                   -0.666829                    0.077246   
1                   -0.858595                    1.408443   
2                    1.958381                    0.786612   
3                   -0.436708                    0.122288   
4                   -0.513498                   -2.525956   

   channel488-1_cls_feature_10  channel488-1_cls_feature_100  \
0                     1.110194                      1.462624   
1                     1.577715                     -0.405029   
2                     1.776155                     -0.303077   
3                    -2.176661                     -0.468790   
4                    -0.227911                      0.828312   

   channel488-1_cls_feature_101  channel488-1_cls_feature_102  \
0                     -0.910066                      0.778230   
1                      0.215081                     -0.245164   
2                     -0.951153                     -2.151748   
3                     -0.662295                     -1.651051   
4                     -2.095522                     -0.546587   

   channel488-1_cls_feature_103  channel488-1_cls_feature_104  \
0                      0.334981                      1.220791   
1                     -0.181003                      1.012115   
2                      1.245307                      1.879176   
3                     -0.513276                     -0.091362   
4                      1.670285                      2.350439   

   channel488-1_cls_feature_105  channel488-1_cls_feature_106  \
0                      0.335859                     -1.021192   
1                     -0.113323                     -0.034175   
2                      1.354164                      0.306250   
3                      1.246866                     -0.305827   
4                      0.469381                      0.710802   

   channel488-1_cls_feature_107  channel488-1_cls_feature_108  \
0                      1.148628                      0.840983   
1                      1.390589                     -0.296024   
2                      1.103006                     -0.059669   
3                      1.012502                      0.540721   
4                     -0.458817                     -0.460443   

   channel488-1_cls_feature_109  channel488-1_cls_feature_11  \
0                      0.875354                    -0.349103   
1                     -0.084396                     0.034298   
2                      0.802181                     0.672

In [5]:
# filter the data and drop nan values
print(profiles_df.shape)
# drop nan values in non metadata columns
profiles_df = profiles_df.dropna(
    subset=profiles_df.columns[~profiles_df.columns.str.contains("Meta")]
)
print(profiles_df.shape)

(188065, 1546)
(188065, 1546)


In [6]:
# get the metadata columns
metadata_cols = profiles_df.columns.str.contains("Metadata_")
metadata_df = profiles_df.loc[:, metadata_cols]
features_df = profiles_df.loc[:, ~metadata_cols]

# set the umap parameters
umap = umap.UMAP(
    n_neighbors=15,
    n_components=2,
    metric="euclidean",
    random_state=42,
    min_dist=0.1,
    n_epochs=500,
    learning_rate=1,
)

# fit the umap model
umap.fit(features_df)

# transform the data
umap_transformed = umap.transform(features_df)

# create a dataframe with the transformed data
umap_df = pd.DataFrame(
    umap_transformed, columns=["UMAP0", "UMAP1"], index=features_df.index
)

# combine the metadata and umap dataframes
umap_df = pd.concat([metadata_df, umap_df], axis=1)
print(umap_df.shape)
umap_df.head()

/home/lippincm/miniforge3/envs/timelapse_analaysis_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/lippincm/miniforge3/envs/timelapse_analaysis_env/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/home/lippincm/miniforge3/envs/timelapse_analaysis_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


(188065, 12)


,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_ImageNumber,Metadata_Nuclei_Number_Object_Number,Metadata_original_index,Metadata_compound,Metadata_dose,Metadata_control,UMAP0,UMAP1
0,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,102,111256,Staurosporine,0.0,negative,-0.436647,5.176337
1,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,106,111257,Staurosporine,0.0,negative,-0.130687,5.963014
2,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,121,111258,Staurosporine,0.0,negative,0.118699,5.211871
3,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,127,111259,Staurosporine,0.0,negative,-4.365322,6.382474
4,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,13,111241,Staurosporine,0.0,negative,1.803694,7.020759


In [7]:
# save the umap dataframe
umap_df.to_parquet(f"../../data/umap/{data_mode}_umap_transformed.parquet")